In [ ]:
import io
import keras
import tensorflow as tf
import os
import numpy as np # used for handling numbers
import pandas as pd # used for handling the dataset
df= pd.read_csv('drive/My Drive/Colab Notebooks/dataset/labeled_outs.txt',delimiter=",",header=None)
#test_df=pd.read_csv('drive/My Drive/Colab Notebooks/dataset/test_outs.txt',delimiter=",",header=None)
# Dataset is now stored in a Pandas Dataframe

In [ ]:
file = open('drive/My Drive/Colab Notebooks/dataset/vocab.txt', 'r')
# read all text
vocab = file.readlines()
vocabulary={}
for w in vocab:
  vocabulary[w.strip()]=""
# close the file
file.close()

In [ ]:
df.columns=['label','text']
df.text=df.text.astype(str)
df.label=df.label.astype(str)
df=df.sample(frac=1)
docs=df['text'].values
labels=df['label'].values

In [ ]:
docs_filtered=[]
for d in docs:
  # split into tokens by white space
  tokens = d.split()
  temp=""
  for t in tokens:
    if t in vocabulary:
      temp=temp+" "+t
  docs_filtered.append(temp)


In [ ]:
# integer encode the documents
from keras.preprocessing.text import one_hot
vocab_size = len(vocabulary)+1

encoded_docs = [one_hot(d, vocab_size) for d in docs_filtered]

In [ ]:
# pad documents to a max length of 4 words
max_length = 200
padded_docs = keras.preprocessing.sequence.pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(len(padded_docs))

In [ ]:
#import sys
#import numpy
#numpy.set_printoptions(threshold=sys.maxsize)
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(labels)
encoded_Y = encoder.transform(labels)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
print(dummy_y.shape)


In [ ]:
emb_dim=32
emb_model = tf.keras.Sequential([ 
    tf.keras.layers.Input(shape=(max_length,)) ,                           
    tf.keras.layers.Embedding(vocab_size, output_dim=emb_dim),
    tf.keras.layers.SimpleRNN(128,return_sequences=True,dropout=0.3),
    tf.keras.layers.SimpleRNN(64,return_sequences=True,dropout=0.2),
    tf.keras.layers.SimpleRNN(32,return_sequences=True,dropout=0.1),
    tf.keras.layers.SimpleRNN(16),
    tf.keras.layers.Dense(8, activation="softmax")
])

emb_model.compile(optimizer='adam',loss= 'categorical_crossentropy',metrics=['acc'])
print(emb_model.summary())


In [ ]:
emb_model.fit(padded_docs, dummy_y,epochs=30,batch_size=256, verbose=1,validation_split=0.1)


In [ ]:
emb_model1 = tf.keras.Model(inputs=emb_model.inputs , outputs=emb_model.layers[3].output)
emb_X = emb_model1.predict(padded_docs)
print(emb_X.shape)
emb_X=np.reshape(emb_X, (emb_X.shape[0],1,-1))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(emb_X, dummy_y, test_size=0.18)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


In [ ]:
model = tf.keras.Sequential([
#    tf.keras.layers.Embedding(vocab_size, 32),
    tf.keras.layers.SimpleRNN(2048,input_shape=(X_train.shape[1],X_train.shape[2]),return_sequences=True,dropout=0.4,recurrent_dropout=0.3),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.SimpleRNN(1024,return_sequences=True,dropout=0.4,recurrent_dropout=0.3),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.SimpleRNN(512,return_sequences=True,dropout=0.3,recurrent_dropout=0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.SimpleRNN(256,return_sequences=True,dropout=0.2,recurrent_dropout=0.2),
    tf.keras.layers.SimpleRNN(64,return_sequences=True),
    tf.keras.layers.SimpleRNN(32),
    tf.keras.layers.Dense(8, activation="softmax")
])

In [ ]:

model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['acc'])
# summarize the model
print(model.summary())

In [ ]:
history = model.fit(X_train, y_train, epochs=25,batch_size=256, shuffle=False, verbose=1,validation_split=0.1)
model.save('saved_model/text_classifier_rnn_word_embedding_epoch_20')

In [ ]:
results = model.evaluate(X_test, y_test)
print(results)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
"""
import numpy as np
array1=[[ 0,  1,  2,  3,  4],
       [ 0,  1,  2,  3,  4],
       [5, 6, 7, 1, 0],
       [5, 6, 7, 1, 0]]
array2=[[ 5, 6, 7, 1, 0],
       [ 0,  1,  2,  3,  4],
       [0,  1,  2,  3,  4],
       [5, 6, 7, 1, 0]]
"""
no_cat=len(y_test[0])
conf_mat=np.zeros(no_cat*no_cat).reshape(no_cat,no_cat)
test_size=len(y_test)
print(conf_mat.shape)
for i in range(test_size):
  y_t=y_test[i]
  y_p=y_pred[i]
  class_t=np.where(y_t == np.amax(y_t))
  class_t=class_t[0][0]
  class_p=np.where(y_p == np.amax(y_p))
  class_p=class_p[0][0]
  conf_mat[class_t][class_p]=conf_mat[class_t][class_p]+1

In [ ]:
tp=np.zeros(no_cat)
fp=np.zeros(no_cat)
tn=np.zeros(no_cat)
fn=np.zeros(no_cat)

prec=np.zeros(no_cat)
rec=np.zeros(no_cat)
f_score=np.zeros(no_cat)

for i in range(no_cat):
  for j in range(no_cat):
    if i==j:
      tp[i]=conf_mat[i][j]
    if i!= j:
      fp[i] = fp[i]+conf_mat[j][i]
      fn[i] = fn[i]+conf_mat[i][j]
    for k in range(no_cat):
      if i!=k & j!=k:
        tn[i]=tn[i]+conf_mat[j][k]

for i in range(no_cat):
    prec[i] = tp[i] * 100.0 / (tp[i] + fp[i])
    rec[i] = tp[i] * 100.0 / (tp[i] + fn[i])
    f_score[i] = 2 * prec[i] * rec[i] / (prec[i] + rec[i])

print(np.average(prec))
print(np.average(rec))
print(np.average(f_score))